In [ ]:
!pip install vecstack

  Created wheel for vecstack: filename=vecstack-0.4.0-cp37-none-any.whl size=19877 sha256=66d99e46655ac460f23b68adf29bcb1323709306c20e88bfb27fe9920dc934dd
  Stored in directory: /root/.cache/pip/wheels/5f/bb/4e/f6488433d53bc0684673d6845e5bf11a25240577c8151c140e
Successfully built vecstack


In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns

In [ ]:
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV,  cross_val_score
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, StackingClassifier
from sklearn import metrics
from sklearn.svm import LinearSVC, SVC

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense,Flatten,Embedding,Activation, Dropout, SimpleRNN, LSTM, Bidirectional,Input
from tensorflow.keras.layers import Conv1D, MaxPooling1D, GlobalMaxPooling1D 

from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator # Generates batches for sequence data

from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam, RMSprop
from tensorflow.keras.regularizers import l1_l2, l2
from tensorflow.keras.optimizers.schedules import ExponentialDecay
from tensorflow.keras import layers
from sklearn.metrics import confusion_matrix, accuracy_score, f1_score, roc_auc_score

from xgboost import XGBClassifier

from vecstack import StackingTransformer

In [ ]:
from google.colab import drive
drive.mount("/content/gdrive")

Mounted at /content/gdrive


In [ ]:
## Full features
X_train_full = pd.read_pickle('/content/gdrive/My Drive/BT4222/Codes/Cleaned Input Data/Full Text SCAM/X_train_fSC.pkl')
X_test_full = pd.read_pickle('/content/gdrive/My Drive/BT4222/Codes/Cleaned Input Data/Full Text SCAM/X_test_fSC.pkl')

## target label
y_train = pd.read_pickle('/content/gdrive/My Drive/BT4222/Codes/Cleaned Input Data/Full Text SCAM/y_train_fSC.pkl')
y_test = pd.read_pickle('/content/gdrive/My Drive/BT4222/Codes/Cleaned Input Data/Full Text SCAM/y_test_fSC.pkl')

## TF-IDF with top 15 features
X_train_tfidf = pd.read_pickle('/content/gdrive/My Drive/BT4222/Codes/Cleaned Input Data/TFIDF SCAM/X_train_tfSC.pkl')
X_test_tfidf = pd.read_pickle('/content/gdrive/My Drive/BT4222/Codes/Cleaned Input Data/TFIDF SCAM/X_test_tfSC.pkl')

## TF-IDF GloVe with top 15 features
X_train_tfidf_glove = pd.read_pickle('/content/gdrive/My Drive/BT4222/Codes/Cleaned Input Data/TFIDF Glove SCAM/X_train_tfglSC.pkl')
X_test_tfidf_glove = pd.read_pickle('/content/gdrive/My Drive/BT4222/Codes/Cleaned Input Data/TFIDF Glove SCAM/X_test_tfglSC.pkl')

## TF-IDF FastText (cc) with top 15 features
X_train_tfidf_cc = pd.read_pickle('/content/gdrive/My Drive/BT4222/Codes/Cleaned Input Data/TFIDF CC SCAM/X_train_tfccSC.pkl')
X_test_tfidf_cc = pd.read_pickle('/content/gdrive/My Drive/BT4222/Codes/Cleaned Input Data/TFIDF CC SCAM/X_test_tfccSC.pkl')

## infersent
X_train_infersent = pd.read_pickle('/content/gdrive/My Drive/BT4222/Codes/Cleaned Input Data/InferSent SCAM/X_train_infSC.pkl')
X_test_infersent = pd.read_pickle('/content/gdrive/My Drive/BT4222/Codes/Cleaned Input Data/InferSent SCAM/X_test_infSC.pkl')
y_train_infersent = pd.read_pickle('/content/gdrive/My Drive/BT4222/Codes/Cleaned Input Data/InferSent SCAM/y_train_infSC.pkl')
y_test_infersent = pd.read_pickle('/content/gdrive/My Drive/BT4222/Codes/Cleaned Input Data/InferSent SCAM/y_test_infSC.pkl')

In [ ]:
svc_tfidf_cc = LinearSVC(penalty = 'l2', max_iter =10000, class_weight = None, C = 0.01)
svc_tfidf = LinearSVC(penalty = 'l2', max_iter =10000, class_weight = 'balanced' , C = 1)
svc_tfidf_glove = LinearSVC(penalty = 'l2', max_iter =10000, class_weight = None, C = 0.1)

xgb_tfidf = XGBClassifier(colsample_bytree = 0.5172645818368209, early_stopping_round = 16, learning_rate = 0.1, max_depth = 7, 
 min_child_weight= 1, n_estimators =564, objective = 'binary:logistic', subsample = 0.6196385276582972)
xgb_tfidf_glove = XGBClassifier(colsample_bytree = 0.5368861000040417, early_stopping_round = 31, learning_rate = 0.1, 
                                 max_depth = 5, min_child_weight = 1, n_estimators = 891, objective = 'binary:logitraw', subsample = 0.3136961083799656)
xgb_tfidf_cc = XGBClassifier(colsample_bytree = 0.5172645818368209, early_stopping_round = 16, learning_rate = 0.1, max_depth = 7,
 min_child_weight = 1, n_estimators = 564, objective = 'binary:logistic', subsample = 0.6196385276582972)

lg_tfidf = LogisticRegression(solver = 'newton-cg', max_iter = 30000, class_weight = None, C = 10)
lg_tfidf_glove = LogisticRegression(solver = 'liblinear', max_iter = 30000, class_weight = None, C = 10)
lg_tfidf_cc = LogisticRegression(solver = 'newton-cg', max_iter = 30000, class_weight = None, C = 1)

In [ ]:
def predict(model, X_test, y_test):
    accuracy = model.score(X_test, y_test)
    y_pred = model.predict(X_test)
    #print("AUC {}".format(metrics.roc_auc_score(y_test, y_pred_prob)))    
    #print("Recall: {}".format(recall_score(y_test, y_pred_class, average='weighted')))
    #print("Precision: {}".format(precision_score(y_test, y_pred_class, average='weighted')))
    tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
    fpr = fp/(fp+tn)
    tpr = tp/(tp + fn)
    #print('False Positive Rate: {}'.format(fp/(fp+tn)))
    f1 = f1_score(y_test, y_pred)
    return {'Accuracy' : accuracy, 'FPR' : fpr, 'Sensitivity' : tpr, 'F1 Score' : f1}

TFIDF

In [ ]:
estimators = [
     ('xgboost', xgb_tfidf),
     ('svc', svc_tfidf)]

# Initialize StackingTransformer
stack = StackingTransformer(estimators, verbose=2, regression = False)

# Fit
stack = stack.fit(X_train_tfidf, y_train)

# Get your stacked features
S_train = stack.transform(X_train_tfidf)
S_test = stack.transform(X_test_tfidf)

lg_tfidf.fit(S_train, y_train)

predict(lg_tfidf, S_test, y_test)

task:         [classification]
n_classes:    [2]
metric:       [accuracy_score]
variant:      [A]
n_estimators: [2]

estimator  0: [xgboost: XGBClassifier]


/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:296: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  FutureWarning


    fold  0:  [0.98421053]
    fold  1:  [0.97368421]
    fold  2:  [0.97894737]
    fold  3:  [0.97625330]
    ----
    MEAN:     [0.97827385] + [0.00390017]

estimator  1: [svc: LinearSVC]
    fold  0:  [0.97631579]
    fold  1:  [0.97368421]
    fold  2:  [0.98157895]
    fold  3:  [0.97625330]
    ----
    MEAN:     [0.97695806] + [0.00287141]

Train set was detected.
Transforming...

estimator  0: [xgboost: XGBClassifier]
    model from fold  0: done
    model from fold  1: done
    model from fold  2: done
    model from fold  3: done
    ----
    DONE

estimator  1: [svc: LinearSVC]
    model from fold  0: done
    model from fold  1: done
    model from fold  2: done
    model from fold  3: done
    ----
    DONE

Transforming...

estimator  0: [xgboost: XGBClassifier]
    model from fold  0: done
    model from fold  1: done
    model from fold  2: done
    model from fold  3: done
    ----
    DONE

estimator  1: [svc: LinearSVC]
    model from fold  0: done
    model from fo

{'Accuracy': 0.9786381842456608,
 'F1 Score': 0.9834710743801653,
 'FPR': 0.03018867924528302,
 'Sensitivity': 0.9834710743801653}

array([[0, 0],
       [1, 1],
       [1, 1],
       ...,
       [0, 0],
       [1, 1],
       [1, 1]])

GLOVE-TFIDF

In [ ]:
estimators = [
     ('xgboost', xgb_tfidf_glove),
     ('svc', svc_tfidf_glove)]

# Initialize StackingTransformer
stack = StackingTransformer(estimators, verbose=2)

# Fit
stack = stack.fit(X_train_tfidf_glove, y_train)

# Get your stacked features
S_train = stack.transform(X_train_tfidf_glove)
S_test = stack.transform(X_test_tfidf_glove)

lg_tfidf_glove.fit(S_train, y_train)

predict(lg_tfidf_glove, S_test, y_test)

task:         [regression]
metric:       [mean_absolute_error]
variant:      [A]
n_estimators: [2]

estimator  0: [xgboost: XGBClassifier]


/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:296: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  FutureWarning


    fold  0:  [0.01052632]
    fold  1:  [0.01052632]
    fold  2:  [0.01315789]
    fold  3:  [0.02374670]
    ----
    MEAN:     [0.01448931] + [0.00545167]

estimator  1: [svc: LinearSVC]
    fold  0:  [0.01315789]
    fold  1:  [0.02631579]
    fold  2:  [0.01052632]
    fold  3:  [0.02374670]
    ----
    MEAN:     [0.01843668] + [0.00672154]

Train set was detected.
Transforming...

estimator  0: [xgboost: XGBClassifier]
    model from fold  0: done
    model from fold  1: done
    model from fold  2: done
    model from fold  3: done
    ----
    DONE

estimator  1: [svc: LinearSVC]
    model from fold  0: done
    model from fold  1: done
    model from fold  2: done
    model from fold  3: done
    ----
    DONE

Transforming...

estimator  0: [xgboost: XGBClassifier]
    model from fold  0: done
    model from fold  1: done
    model from fold  2: done
    model from fold  3: done
    ----
    DONE

estimator  1: [svc: LinearSVC]
    model from fold  0: done
    model from fo

{'Accuracy': 0.9786381842456608,
 'F1 Score': 0.9834368530020703,
 'FPR': 0.026415094339622643,
 'Sensitivity': 0.981404958677686}

FastText-TFIDF

In [ ]:
estimators = [
     ('xgboost', xgb_tfidf_cc),
     ('svc', svc_tfidf_cc)]

# Initialize StackingTransformer
stack = StackingTransformer(estimators, verbose=2)

# Fit
stack = stack.fit(X_train_tfidf_cc, y_train)

# Get your stacked features
S_train = stack.transform(X_train_tfidf_cc)
S_test = stack.transform(X_test_tfidf_cc)

lg_tfidf_cc.fit(S_train, y_train)

predict(lg_tfidf_cc, S_test, y_test)

task:         [regression]
metric:       [mean_absolute_error]
variant:      [A]
n_estimators: [2]

estimator  0: [xgboost: XGBClassifier]


/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:296: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  FutureWarning


    fold  0:  [0.00526316]
    fold  1:  [0.00789474]
    fold  2:  [0.00263158]
    fold  3:  [0.01319261]
    ----
    MEAN:     [0.00724552] + [0.00390537]

estimator  1: [svc: LinearSVC]
    fold  0:  [0.00789474]
    fold  1:  [0.01315789]
    fold  2:  [0.01315789]
    fold  3:  [0.01319261]
    ----
    MEAN:     [0.01185078] + [0.00228407]

Train set was detected.
Transforming...

estimator  0: [xgboost: XGBClassifier]
    model from fold  0: done
    model from fold  1: done
    model from fold  2: done
    model from fold  3: done
    ----
    DONE

estimator  1: [svc: LinearSVC]
    model from fold  0: done
    model from fold  1: done
    model from fold  2: done
    model from fold  3: done
    ----
    DONE

Transforming...

estimator  0: [xgboost: XGBClassifier]
    model from fold  0: done
    model from fold  1: done
    model from fold  2: done
    model from fold  3: done
    ----
    DONE

estimator  1: [svc: LinearSVC]
    model from fold  0: done
    model from fo

{'Accuracy': 0.986648865153538,
 'F1 Score': 0.9896480331262941,
 'FPR': 0.01509433962264151,
 'Sensitivity': 0.987603305785124}